# StrongholdNet: Train an RNN (LSTM) to navigate through a Stronghold

The idea is that we interpret the (shortest) path from any room in the stronghold to the portal room as *sequential data* that we feed to an RNN.

In [2]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras as K
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from dataset_rnn import parse_tree_generator, print_stronghold_tree
from anytree import Node, RenderTree, Walker
from anytree.search import find_by_attr
import random

In [3]:
df = pd.read_csv('100k_dataset_rnn.csv', delimiter=' ')

In [4]:
df.head(24)

,stronghold,room,downwards,orientation,parent_room,parent_exit,child_room_1,child_room_2,child_room_3,child_room_4,child_room_5,exit
0,0,RightTurn,0,S,Stairs,1,Stairs,None,None,None,None,0
1,0,Stairs,0,S,Corridor,2,RightTurn,None,None,None,None,0
2,0,Corridor,0,W,FiveWayCrossing,1,Corridor,Stairs,RightTurn,None,None,1
3,0,Corridor,1,W,Corridor,1,LeftTurn,Corridor,None,None,None,2
4,0,Corridor,1,S,Corridor,2,Corridor,None,None,None,None,1
5,0,Corridor,1,S,Corridor,1,RightTurn,None,None,None,None,1
6,0,RightTurn,1,S,Corridor,1,PortalRoom,None,None,None,None,1
7,1,PrisonHall,0,N,SquareRoom,1,Corridor,None,None,None,None,0
8,1,SquareRoom,0,N,ChestCorridor,1,PrisonHall,RightTurn,ChestCorridor,None,None,0
9,1,ChestCorridor,0,N,RightTurn,1,SquareRoom,None,None,None,None,0


The problem is that we have to structure our data such that in each time step the model gets to know where we went in the last step.

In [5]:
# one-hot encode
cols = [
        'room',
        'downwards',
        'orientation',
        'parent_room',
        'parent_exit',
        'child_room_1',
        'child_room_2',
        'child_room_3',
        'child_room_4',
        'child_room_5']
onehot = ColumnTransformer([("one-hot", OneHotEncoder(), cols)], remainder='passthrough')
onehot.fit(df)
df_onehot = pd.DataFrame(onehot.transform(df).toarray(), index=df.index, columns=pd.Index(onehot.get_feature_names()))

We need to group our sequences by stronghold.

In [44]:
def sequencelify(df):
    for window in range(1, 10):
        for _, s in df.groupby('stronghold'):
            # each stronghold is a batch
            X_batch = []
            y_batch = []
            s = s.drop('stronghold', axis=1).to_numpy()
            for t in range(s.shape[0] - window + 1):
                if s.shape[0] < window:
                    continue
                X = s[t:t+window, :-1]
                X = X.reshape(window, X.shape[1])
                y = np.array(s[t+window-1, -1])
                X_batch.append(X)
                y_batch.append(y)
            X_batch = np.array(X_batch)
            y_batch = np.array(y_batch)
            X_batch = X_batch.reshape(s.shape[0] - window + 1, window, s.shape[1] - 1)
            y_batch = y_batch.reshape(s.shape[0] - window + 1, 1)
            y_batch = K.utils.to_categorical(y_batch, num_classes=6)
            yield X_batch, y_batch

In [46]:
# just checking if generator works
count = 0
for x, y in sequencelify(df_train):
    print(x)
    print(y)
    break

[[[0. 0. 0. 0. 0. 1. 0. 0. 0. 1. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.
   1. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0.
   0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.
   0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.
   0. 0. 0.]]

 [[0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 0. 0. 0. 1. 0. 0. 1. 0. 0. 0. 0. 0. 0.
   0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.
   0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.
   0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.
   0. 0. 0.]]

 [[0. 1. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 1. 0. 0. 0. 0. 0.
   0. 0. 1. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
   0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.
   0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.
   0. 0. 0.]]

 [[0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 1. 0. 1. 0. 0. 0. 0. 0. 0.
   0. 0. 1. 0. 0. 0

In [47]:
df_train, df_test = train_test_split(
        df_onehot,
        test_size=0.1,
        random_state=1337,
        shuffle=False)

In [48]:
df_train.head()

,one-hot__x0_ChestCorridor,one-hot__x0_Corridor,one-hot__x0_FiveWayCrossing,one-hot__x0_LeftTurn,one-hot__x0_PrisonHall,one-hot__x0_RightTurn,one-hot__x0_SpiralStaircase,one-hot__x0_SquareRoom,one-hot__x0_Stairs,one-hot__x1_0,...,one-hot__x9_None,one-hot__x9_PortalRoom,one-hot__x9_PrisonHall,one-hot__x9_RightTurn,one-hot__x9_SmallCorridor,one-hot__x9_SpiralStaircase,one-hot__x9_SquareRoom,one-hot__x9_Stairs,stronghold,exit
0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0
4,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [49]:
print("df_train:", df_train.shape)
print("df_test:", df_test.shape)

df_train: (631518, 97)
df_test: (70169, 97)


In [50]:
n_samples = df_train.shape[0]
n_features = df_train.shape[1] - 2

In [51]:
model = K.Sequential()
model.add(K.layers.LSTM(
        64,
        batch_input_shape=(None, None, n_features),
        return_sequences=False,
        stateful=False))
model.add(K.layers.Dense(
        6,
        activation='softmax'))
model.compile(
        optimizer='adam',
        loss='categorical_crossentropy',
        metrics=['accuracy'])
model.summary()

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_4 (LSTM)                (None, 64)                40960     
_________________________________________________________________
dense_10 (Dense)             (None, 6)                 390       
Total params: 41,350
Trainable params: 41,350
Non-trainable params: 0
_________________________________________________________________


In [52]:
model.fit(
        sequencelify(df_train),
        epochs=10,
        steps_per_epoch=10000)

Epoch 1/10
10000/10000 [==============================] - 10s 973us/step - loss: 0.4581 - accuracy: 0.7953
Epoch 2/10
10000/10000 [==============================] - 11s 1ms/step - loss: 0.4189 - accuracy: 0.8042
Epoch 3/10
10000/10000 [==============================] - 10s 1ms/step - loss: 0.4153 - accuracy: 0.8072
Epoch 4/10
10000/10000 [==============================] - 10s 1ms/step - loss: 0.4099 - accuracy: 0.8110
Epoch 5/10
10000/10000 [==============================] - 10s 1ms/step - loss: 0.4078 - accuracy: 0.8122
Epoch 6/10
10000/10000 [==============================] - 10s 985us/step - loss: 0.4073 - accuracy: 0.8133
Epoch 7/10
10000/10000 [==============================] - 10s 982us/step - loss: 0.4033 - accuracy: 0.8140
Epoch 8/10
10000/10000 [==============================] - 10s 988us/step - loss: 0.4036 - accuracy: 0.8133
Epoch 9/10
10000/10000 [==============================] - 10s 1ms/step - loss: 0.4072 - accuracy: 0.8133
Epoch 10/10
10000/10000 [======================

In [53]:
model.evaluate(sequencelify(df_test))

90333/90333 [==============================] - 79s 876us/step - loss: 0.2876 - accuracy: 0.8609


[0.2876206934452057, 0.860894501209259]

Save/load model weights.

In [54]:
model.save("rnn_2.keras")
#model = K.models.load_model("rnn_2.keras")

STUFF BELOW IS STILL BROKEN

More complex evaluation - does not work yet!

In [57]:
g = parse_tree_generator('100k_strongholds_test.txt')
root = next(g)

In [ ]:
def evaluate_nav(room: Node, model: K.Model, onehot: OneHotEncoder, n_rooms=0):
    if len([ v for v in room.children if v.name == 'PortalRoom' ]) > 0:
        return n_rooms
    
    if n_rooms > 30:
        #print("giving up")
        return False
    
    if room.name in ['Library', 'SmallCorridor']:
        #print("bad nav")
        return False
    
    #print("room", room.name)
    x = pd.DataFrame([(
            room.name,
            room.orientation,
            room.parent.name,
            room.exit,
            *([c.name for c in room.children] + ['None'] * (5 - len(room.children))), -1)],
            columns=df.columns)
    x_onehot = pd.DataFrame(onehot.transform(x).toarray(), columns=pd.Index(onehot.get_feature_names()))
    x_onehot.drop('exit', axis=1, inplace=True)
    x_np = x_onehot.to_numpy().reshape(1, 1, n_features)
    y_hat = model.predict(x_np)
    exit_hat = y_hat.argmax(axis=-1)[0]
    #print("exit_hat", exit_hat)
    
    return evaluate_nav([room.parent, *room.children][exit_hat], model, onehot, n_rooms + 1)

total = 0
hits = 0
for root in parse_tree_generator('100k_strongholds_test.txt'):
    #print_stronghold_tree(root)
    root = root.children[0]sequencelify(df_train, 8)
    n_rooms = evaluate_nav(root, model, onehot)
    #(upwards, common, downwards) = Walker().walk(root, find_by_attr(root, 'PortalRoom'))
    #delta = n_rooms - len(downwards)
    total += 1
    if n_rooms != False:
        hits += 1
        print("n_rooms:", n_rooms)
        print("total:", total)
        print("hits:", hits)
        print("ratio:", hits / total)
        print()